# CUDA - co to jest?

CUDA to platforma obliczeń równoległych i model programowania, które sprawiają, że używanie GPU do obliczeń ogólnego przeznaczenia staje się proste i eleganckie. 

Programista pisze w znanym sobie języku, takim jak C, C++, Python czy coraz szersza lista wspieranych języków, a jedynie wykorzystuje rozszerzenia tych języków w postaci kilku podstawowych słów kluczowych.

Dzięki tym słowom kluczowym programista może wyrazić ogromny poziom równoległości i wskazać kompilatorowi fragment aplikacji, który ma zostać odwzorowany na GPU.

# TPU EXAMPLE - COLAB

## Kroki w Colab

1. Włącz **TPU** w Colab:
   `Runtime → Change runtime type → Hardware accelerator → TPU`

2. Uruchom poniższy kod w komórkach:

```python
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

# --- konfiguracja TPU ---
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # znajdź TPU
    print("Urządzenie TPU:", tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
except ValueError:
    print("TPU niedostępne, używam CPU/GPU")
    strategy = tf.distribute.get_strategy()
```

---

## Ładowanie danych MNIST

```python
# Dane: ręcznie pisane cyfry 28x28
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalizacja
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# Dodanie wymiaru kanału (28,28) -> (28,28,1)
x_train = x_train[..., None]
x_test = x_test[..., None]

print("Shape train:", x_train.shape, "Shape test:", x_test.shape)
```

---

## Definicja modelu na TPU

> dzięki `strategy.scope()` model i optymalizator działają na TPU

```python
with strategy.scope():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, 3, activation="relu", input_shape=(28,28,1)),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Conv2D(64, 3, activation="relu"),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(10, activation="softmax")
    ])

    model.compile(
        optimizer="adam",
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    model.fit(x_train, y_train, epochs=5, batch_size=128, validation_split=0.1)
    model.evaluate(x_test, y_test)
```

---

## Trenowanie modelu

```python
model.fit(x_train, y_train, epochs=5, batch_size=128, validation_split=0.1)
```

---

## Ewaluacja

```python
model.evaluate(x_test, y_test)
```

---

✨ Ten kod powinien uruchomić trening CNN na **TPU** w Colab. Będzie zauważalnie szybszy niż na CPU, a czasami szybszy niż GPU.
